In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('Silq.db')
cursor = conn.cursor()

In [4]:
sql1 = """
select 
	strftime('%Y-%m', SignUpDate) AS CohertMonth, 
	count(distinct UserID) as NumberOfUsers 
from Users GROUP BY 1;
"""



cohert_user_count = pd.read_sql_query(sql1, conn)

In [5]:
cohert_user_count

,CohertMonth,NumberOfUsers
0,2021-03,1
1,2021-04,3
2,2021-05,3
3,2021-09,2
4,2021-12,1


In [5]:
cohert_user_count.to_csv('/home/antash/silq/Silq Report/signupCount.csv')

In [15]:
sql2 = """
WITH FirstOrderMonth AS (
    SELECT 
        UserID, 
        MIN(strftime('%Y-%m', OrderDate)) AS FirstOrderMonth
    FROM 
        Orders
    GROUP BY 
        UserID
),

UserSignUpFirstOrder AS (
    SELECT 
        u.UserID, 
        strftime('%Y-%m', u.SignUpDate) AS SignupMonth,
		f.FirstOrderMonth
    FROM 
        Users u
	LEFT JOIN FirstOrderMonth f on u.UserID = f.UserID
),

SignUpOrderMonth AS (
	SELECT
		SignupMonth,
		RetentionMonth,
		min(FirstOrderMonth) as FirstOrderMonth
	FROM UserSignUpFirstOrder 
	CROSS JOIN (
		select 'M0' as RetentionMonth
		UNION
		SELECT 'M1' as RetentionMonth
		UNION
		SELECT 'M2' as RetentionMonth
		)
	group by 1,2
),

UserActivity AS (
    SELECT 
        sc.UserID,
        sc.SignupMonth,
        fo.FirstOrderMonth,
        strftime('%Y-%m', o.OrderDate) AS OrderMonth
    FROM 
        UserSignUpFirstOrder sc
    JOIN 
        SignUpOrderMonth fo ON sc.SignUpMonth = fo.SignUpMonth
    JOIN 
        Orders o ON sc.UserID = o.UserID
	
	group by 1,2,3,4
),

prelim as (
select *, 
	case WHEN FirstOrderMonth = OrderMonth then 'M0'
		when OrderMonth = strftime('%Y-%m', datetime(FirstOrderMonth || '-01', '+1 month')) then 'M1'
		when OrderMonth = strftime('%Y-%m', datetime(FirstOrderMonth || '-01', '+2 months')) then 'M2'
	else NULL end as RetentionMonth

from UserActivity ORDER BY SignUpMonth
),

RetentionUser as (
SELECT *,count(RetentionMonth) OVER (partition by SignUpMonth, RetentionMonth) as UserCount from prelim
),

Main as (
SELECT s.SignupMonth, s.RetentionMonth, coalesce(r.UserCount,0) as ActiveUserCount, count(distinct u.UserID) as TotalUserCount
from SignUpOrderMonth s 
LEFT JOIN RetentionUser r ON s.SignupMonth = r.SignupMonth and s.RetentionMonth = r.RetentionMonth
LEFT JOIN UserSignUpFirstOrder u ON s.SignUpMonth = u.SignUpMonth
group by 1,2,3 
)


select *, ROUND((ROUND(ActiveUserCount,2)/ROUND(TotalUserCount,2))*100,0) as RetentionRate from Main
"""

In [16]:
cohert_retained_count_per = pd.read_sql_query(sql2, conn)

In [17]:
cohert_retained_count_per

,SignupMonth,RetentionMonth,ActiveUserCount,TotalUserCount,RetentionRate
0,2021-03,M0,1,1,100.0
1,2021-03,M1,0,1,0.0
2,2021-03,M2,0,1,0.0
3,2021-04,M0,1,3,33.0
4,2021-04,M1,0,3,0.0
5,2021-04,M2,2,3,67.0
6,2021-05,M0,1,3,33.0
7,2021-05,M1,0,3,0.0
8,2021-05,M2,0,3,0.0
9,2021-09,M0,2,2,100.0


In [18]:
cohert_retained_count_per.to_csv('/home/antash/silq/Silq Report/RetainedCount.csv', index= False)